This notebook collects static attributes (basin length, basin area, reach length) for all of our basins of interest from the NWM 3.0 hydrofabric. 

- `hfv3_conuscats.parquet` and `hfv3_conusreaches.parquet` adapted from https://water.noaa.gov/resources/downloads/nwm/NWM_channel_hydrofabric.tar.gz
- `camels_upstream_file` derived from `../02_get_upstream_basins/output/camels_upstream_dict.json`.

Written by Quinn Lee (GitHub @quinnylee)

In [1]:
import pandas as pd
import json

In [2]:
# Read hydrofabric information
conuscats = pd.read_parquet("../hfv3_conuscats.parquet")
conusreaches = pd.read_parquet("../hfv3_conusreaches.parquet")

In [3]:
conuscats.head()

,ID,Shape_Length,Shape_Area,geometry
0,91692,0.054827,0.000073,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
1,85234,0.107728,0.000204,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
2,85214,0.033429,0.000038,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
3,91966,0.038875,0.000045,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...
4,89996,0.023342,0.000014,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...


In [9]:
conuscats['ID'].dtype

dtype('int32')

In [4]:
camelsatts_dict = {"id": [], "basin_length": [], "basin_area": [], "reach_length": []}

In [6]:
with open("../02_get_upstream_basins/output/camels_upstream_dict.json", "r") as camels_upstream_file:
    camels_upstream_dict = json.load(camels_upstream_file)

In [7]:
camels_list = []
for key in camels_upstream_dict.keys():
    camels_list.append(int(key))
    for basin in camels_upstream_dict[key]:
        camels_list.append(basin)

In [10]:
conuscats[conuscats["ID"] == 11049274]["Shape_Length"].values[0]

0.03722843916858033

In [11]:
for camels_basin in camels_list:

    try:
        # Get the basin length
        basin_length = conuscats[conuscats["ID"] == camels_basin]["Shape_Length"].values[0]
        # Get the basin area
        basin_area = conuscats.loc[conuscats["ID"] == camels_basin]["Shape_Area"].values[0]
    except IndexError:
        basin_length = 'NaN'
        basin_area = 'NaN'
    
    try:
        # Get the reach length
        reach_length = conusreaches.loc[conusreaches["ID"] == camels_basin]["Shape_Length"].values[0]
    except IndexError:
        reach_length = 'NaN'

    camelsatts_dict["id"].append(camels_basin)
    camelsatts_dict["basin_length"].append(basin_length)
    camelsatts_dict["basin_area"].append(basin_area)
    camelsatts_dict["reach_length"].append(reach_length)

In [12]:
camelsatts = pd.DataFrame(camelsatts_dict)


In [14]:
camelsatts.to_csv("camelsatts.csv", index=False)
